# Applying Advanced Transformations
- Daniel Barella
- 2/4/23

In [1]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

## The Data
You will be working with a heavily modified version of the Superheroes dataset from Kaggle.

The dataset includes two csv's:

- [superhero_info.csv](https://docs.google.com/spreadsheets/d/e/2PACX-1vS1ZstYLwFgwhZnqDsPjtnlHYhJp_cmW55J8JD5mym0seRsaem3px7QBtuFF0LiI7z1PLCkVKAkdO7J/pub?output=csv):
 - Contains Name, Publisher, Demographic Info, and Body measurements.
- [superhero_powers.csv](https://docs.google.com/spreadsheets/d/e/2PACX-1vSzdWOBaXOoz52vPmCFV5idNlDBohLY1Lsbc1IfZIZQ7cV_aNB2wYBfhF49uE1TaO1B5MQCGWiNrFfd/pub?output=csv):
 - Contains Hero name and list of powers

## The Task
Your task is two-fold:

I. Clean the files and combine them into one final DataFrame.

- This dataframe should have the following columns:
 - Hero (Just the name of the Hero)
 - Publisher
 - Gender
 - Eye color
 - Race
 - Hair color
 - Height (numeric)
 - Skin color
 - Alignment
 - Weight (numeric)
- Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
 - Agility
 - Flight
 - Superspeed
 - etc.

Hint: There is a space in "100 kg" or "52.5 cm"

II. Use your combined DataFrame to answer the following questions.

1. Compare the average weight of super powers who have Super Speed to those who do not.
2. What is the average height of heroes for each publisher?

In [2]:
superhero_info = pd.read_csv('/Users/danielbarella/Downloads/superhero_info - superhero_info.csv')
superhero_info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
superhero_info.dropna(inplace=True)

In [4]:
superhero_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [5]:
superhero_info['Hero|Publisher'].head()

0            A-Bomb|Marvel Comics
1    Abe Sapien|Dark Horse Comics
2              Abin Sur|DC Comics
3       Abomination|Marvel Comics
4     Absorbing Man|Marvel Comics
Name: Hero|Publisher, dtype: object

In [6]:
superhero_info['Hero|Publisher'].str.split('|',expand=True)

,0,1
0,A-Bomb,Marvel Comics
1,Abe Sapien,Dark Horse Comics
2,Abin Sur,DC Comics
3,Abomination,Marvel Comics
4,Absorbing Man,Marvel Comics
...,...,...
458,Yellowjacket,Marvel Comics
459,Yellowjacket II,Marvel Comics
460,Yoda,George Lucas
461,Zatanna,DC Comics


In [7]:
# save the 2 new columns into the dataframe
superhero_info[['Hero','Publisher']] = superhero_info['Hero|Publisher'].str.split('|',expand=True)
superhero_info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [8]:
## drop the original column 
superhero_info = superhero_info.drop(columns=['Hero|Publisher'])
superhero_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [9]:
superhero_info['Measurements'].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [10]:
# Replace single quotes with double quotes
superhero_info['Measurements'] = superhero_info['Measurements'].str.replace("'", '"')

# Convert string representations of dictionaries to actual dictionaries
superhero_info['Measurements'] = superhero_info['Measurements'].apply(json.loads)

# Extract height and weight values into separate columns
superhero_info['Height'] = superhero_info['Measurements'].apply(lambda x: x['Height'])
superhero_info['Weight'] = superhero_info['Measurements'].apply(lambda x: x['Weight'])

# Drop the original 'Measurements' column
superhero_info.drop('Measurements', axis=1, inplace=True)


In [11]:
superhero_info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


In [12]:
superhero_powers = pd.read_csv('/Users/danielbarella/Downloads/superhero_powers - superhero_powers.csv')
superhero_powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [13]:
superhero_powers.dropna(inplace=True)

In [14]:
superhero_powers['Powers'].value_counts()

Intelligence                                                                                                                                                                                                                                                         8
Durability,Super Strength                                                                                                                                                                                                                                            5
Agility,Stealth,Marksmanship,Weapons Master,Stamina                                                                                                                                                                                                                  4
Marksmanship                                                                                                                                                                                                       

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convert the "Powers" column into a list of strings
powers = superhero_powers["Powers"].tolist()

# Tokenize each string into individual powers
powers_split = []
for power in powers:
    powers_split.append(power.split(","))

# Create a MultiLabelBinarizer object and fit it to the list of powers
mlb = MultiLabelBinarizer()
powers_onehot = mlb.fit_transform(powers_split)

# Convert the one-hot encoded matrix into a pandas DataFrame
powers_df = pd.DataFrame(powers_onehot, columns=mlb.classes_)

# Concatenate the original DataFrame with the one-hot encoded powers
superhero_powers = pd.concat([superhero_powers, powers_df], axis=1)


In [16]:
merged_df = pd.merge(superhero_info, superhero_powers, left_on='Hero', right_on='hero_names', how='inner')
merged_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg,...,0,0,0,0,0,0,0,0,0,0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg,...,0,0,0,0,0,0,1,0,0,0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg,...,0,0,0,0,0,0,0,0,0,0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg,...,0,0,0,0,0,0,0,0,0,0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg,...,0,0,0,0,0,0,0,0,0,0


In [17]:
merged_df.drop('hero_names', axis=1, inplace=True)

In [18]:
merged_df

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg,...,0,0,0,0,0,0,0,0,0,0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg,...,0,0,0,0,0,0,1,0,0,0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg,...,0,0,0,0,0,0,0,0,0,0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg,...,0,0,0,0,0,0,0,0,0,0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Male,Human,good,Blond,blue,Unknown,Yellowjacket,Marvel Comics,183.0 cm,83.0 kg,...,0,0,0,0,0,0,0,0,0,0
459,Female,Human,good,Strawberry Blond,blue,Unknown,Yellowjacket II,Marvel Comics,165.0 cm,52.0 kg,...,0,0,0,0,0,0,0,0,0,0
460,Male,Yoda's species,good,White,brown,green,Yoda,George Lucas,66.0 cm,17.0 kg,...,0,0,0,0,0,0,1,0,0,0
461,Female,Human,good,Black,blue,Unknown,Zatanna,DC Comics,170.0 cm,57.0 kg,...,0,0,0,0,1,0,0,1,0,0


## II. Use your combined DataFrame to answer the following questions.

1. Compare the average weight of super heros who have Super Speed to those who do not.


In [19]:
merged_df["Weight"].unique()

array(['441.0 kg', '65.0 kg', '90.0 kg', '122.0 kg', '88.0 kg', '81.0 kg',
       '104.0 kg', '108.0 kg', '169.0 kg', '173.0 kg', '101.0 kg',
       '68.0 kg', '57.0 kg', '54.0 kg', '83.0 kg', '86.0 kg', '358.0 kg',
       '135.0 kg', '106.0 kg', '146.0 kg', '63.0 kg', '98.0 kg',
       '270.0 kg', '59.0 kg', '50.0 kg', '126.0 kg', '67.0 kg',
       '180.0 kg', '77.0 kg', '52.0 kg', '61.0 kg', '95.0 kg', '79.0 kg',
       '133.0 kg', '181.0 kg', '216.0 kg', '71.0 kg', '124.0 kg',
       '155.0 kg', '113.0 kg', '58.0 kg', '92.0 kg', '97.0 kg', '56.0 kg',
       '230.0 kg', '495.0 kg', '55.0 kg', '99.0 kg', '158.0 kg',
       '74.0 kg', '116.0 kg', '170.0 kg', '70.0 kg', '225.0 kg',
       '817.0 kg', '27.0 kg', '91.0 kg', '178.0 kg', '383.0 kg',
       '171.0 kg', '187.0 kg', '132.0 kg', '89.0 kg', '110.0 kg',
       '412.0 kg', '306.0 kg', '80.0 kg', '203.0 kg', '96.0 kg',
       '18.0 kg', '45.0 kg', '167.0 kg', '16.0 kg', '630.0 kg',
       '268.0 kg', '62.0 kg', '115.0 kg', '4.0 kg'

In [21]:
# Remove the 'kg' from the weight column and convert it to a float
merged_df["Weight"] = merged_df["Weight"].str.split(".").str[0]

merged_df['Weight'] = merged_df['Weight'].astype(float)

# Add 'Weight (kg)' to the title of the weight column
merged_df = merged_df.rename(columns={"Weight": "Weight (kg)"})

merged_df["Weight (kg)"] = merged_df["Weight (kg)"].apply(lambda x: int(x))

# Calculate the average weight of super heroes with and without super speed
speed_mask = merged_df["Super Speed"] == 1
average_weight_with_speed = merged_df[speed_mask]["Weight (kg)"].mean()
average_weight_without_speed = merged_df[~speed_mask]["Weight (kg)"].mean()

# Print the results
print(f"Average weight of super heroes with Super Speed: {average_weight_with_speed:.2f} kg")
print(f"Average weight of super heroes without Super Speed: {average_weight_without_speed:.2f} kg")

Average weight of super heroes with Super Speed: 129.40 kg
Average weight of super heroes without Super Speed: 101.77 kg


In [22]:
merged_df["Super Speed"].unique()

array([0, 1])

2. What is the average height of heroes for each publisher?

In [23]:
merged_df["Height"] = merged_df["Height"].str.split(".").str[0]

merged_df = merged_df.rename(columns={"Height": "Height (cm)"})

# convert the "Height" column to numerical values
merged_df["Height (cm)"] = merged_df["Height (cm)"].apply(lambda x: int(x))

# group the data by the "Publisher" column and calculate the average height of heroes for each publisher
average_height = merged_df.groupby("Publisher").mean()["Height (cm)"]

# print the result
print(average_height)

Publisher
DC Comics            181.920290
Dark Horse Comics    176.909091
George Lucas         159.600000
Image Comics         211.000000
Marvel Comics        191.545455
Shueisha             171.500000
Star Trek            181.500000
Team Epic TV         180.750000
Unknown              178.000000
Name: Height (cm), dtype: float64


/var/folders/l7/w6kg6nmd3dgfd2wj06wwdb1c0000gn/T/ipykernel_79844/2771465851.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_height = merged_df.groupby("Publisher").mean()["Height (cm)"]


In [24]:
merged_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height (cm),Weight (kg),...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203,441,...,0,0,0,0,0,0,0,0,0,0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191,65,...,0,0,0,0,0,0,1,0,0,0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185,90,...,0,0,0,0,0,0,0,0,0,0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203,441,...,0,0,0,0,0,0,0,0,0,0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193,122,...,0,0,0,0,0,0,0,0,0,0


In [25]:
merged_df.to_csv('applying_advanced_transformations.csv', index=False)